In [1]:
import pandas as pd
from sqlalchemy import create_engine
from time import time
import argparse
import os
import gzip
import shutil
from datetime import timedelta
from pathlib import Path 
import pandas as pd
from prefect import flow, task
from prefect_gcp.cloud_storage import GcsBucket
from prefect_gcp import GcpCredentials


* https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
* https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
* https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
* https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2023-01.parquet
* https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-01.parquet

In [3]:
def extract_from_gcs(color: str, year : int, month : int) -> Path:
    """ Download trip data from gcs bucket"""

    gcs_path = f"../data/{color}/{color}_tripdata_{year}-{month:02}.parquet"

    gcs_block = GcsBucket.load("zoom-gcs")

    gcs_block.get_directory(from_path = gcs_path, local_path = f"../data/")

    return Path(f"../data/{gcs_path}")

In [4]:
color = "yellow"
year = 2021
month = 1


path = extract_from_gcs(color, year, month)
# df = transform(path)
# write_bq(df)

AttributeError: 'coroutine' object has no attribute 'get_directory'